# Normal distribution

In [47]:
from scipy.integrate import quad as integrate
from scipy.special import binom
from scipy.optimize import fsolve as solve
import numpy as np
from numpy import sqrt, sin, cos, pi, exp, inf, round, ceil
from numpy import array as arr

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

def fig():
    return figure(plot_width=950, plot_height=500)

Loading BokehJS ...

Define the normal distribution:

In [3]:
def normal_distribution(mu, sigma):
    def f(x):
        return 1/sqrt(2*pi*sigma**2)*exp(-(x-mu)**2/(2*sigma**2))
    return f

In [4]:
integrate(normal_distribution(12,3), -inf, inf)

(0.9999999999999984, 1.6699038772828793e-09)

Define the binomial distribution:

In [20]:
def binomial_distribution(n):
    def f(x):
        return binom(n,x)*(1/2)**n
    return f

Define our specific cases:

In [21]:
n = 20 # Up until now, only works for even n
mu = int(n/2)

bin = binomial_distribution(n)

sigma = sqrt(sum(bin(x)*(mu-x)**2 for x in range(n+1)))

norm = normal_distribution(mu, sigma)

Some basic plots:

In [24]:
x_norm = np.linspace(-1, n+1, 1000)
y_norm = norm(x_norm)

x_bin = np.linspace(0,n,n+1)
y_bin = bin(x_bin)

In [25]:
p = fig()
p.line(x_norm, y_norm)
p.vbar(x_bin, top=y_bin, width=0.8)
show(p)

### Finding the corresponding intervals
The next step is to find out the intervals of the normal distribution whose integrals yield the same values as the binomial distribution for the corresponding $x$. 

The function ``integ_around`` returns 
$$ \int^{\mu+a}_{\mu-a} f(x) \;dx$$
as a function of $a$ for the given $\mu$ where $f(x)$ is the normal distribution. Optionally fixed upper or lower bounds can be passed.

In [26]:
def integ_around(val, upper_bound=False, lower_bound=False):
    # Create function
    def f(a):
        # Set upper and lower bounds depending on whether they are specified or not
        upper = val+a if not upper_bound else upper_bound
        lower = val-a if not lower_bound else lower_bound
        # Return value of integral
        return integrate(norm, lower, upper)
    return f

The function ``diff_to_integ`` returns a function of $a$ which gives the difference of the bionmial distribution at the given $\mu$ and the integral $ \int^{\mu+a}_{\mu-a} f(x) \;dx$. Here again optionally fixed upper or lower bounds can be passed.

In [27]:
def diff_to_integ(val, upper_bound=False, lower_bound=False):
    integ = integ_around(val, upper_bound, lower_bound)
    def f(a):
        return integ(a) - bin(val)
    return f

Next, we have to find the zeros of ``diff_to_integ`` for different $\mu$.

The interval endpoints are collected in the array ``intervals``. 

In [28]:
intervals = []

# The first
a = solve(diff_to_integ(mu), 0)[0]
intervals.append(mu+a)

# The rest
for val in range(mu+1, n):
    lower_bound = intervals[-1]
    a = solve(diff_to_integ(val, lower_bound=lower_bound), 0)[0]
    intervals.append(val+a)

In [29]:
intervals

[10.497874815138442,
 11.496107575745944,
 12.501972300399022,
 13.521207990540706,
 14.560700324772521,
 15.629378769106374,
 16.73989052048239,
 17.912263047995339,
 19.183902573892187,
 20.650394089291296]

In [30]:
other_side = n - arr(intervals)[::-1]
intervals = np.concatenate((other_side, intervals))

Now, let us take a look at how it looks.

In [31]:
p = fig()
p.line(x_norm, y_norm)
p.vbar(intervals, top=norm(intervals), width=0.02)
show(p)

Seems to be quite sensible. Now, let us change the standard deviation of the normal distribution, but compute the integrals with the same upper and lower bounds respectively.

In [32]:
# "nnorm" for "narrower norm" or "new norm" as one prefers
nnorm = normal_distribution(n/2, 3)

In [33]:
x_nnorm = np.linspace(-1, n+1, 1000)
y_nnorm = nnorm(x_nnorm)

p = fig()
p.line(x_nnorm, y_nnorm)
p.vbar(intervals, top=nnorm(intervals), width=0.02)
show(p)

Let us add $\pm\infty$ to the list of intervals in order for it to be comprehensive:

In [34]:
intervals = list(intervals)
intervals.insert(0, -inf)
intervals.append(inf)

In [35]:
vals = []
for i in range(len(intervals)-1):
    vals.append(integrate(nnorm, intervals[i], intervals[i+1])[0])

In [36]:
p = fig()
p.line(x_nnorm, y_nnorm)
p.vbar(x_bin, top=vals, width=0.8)
show(p)

Whilst this looks reasonable for $\sigma$ not very far from its original value, the more it differs the worse the "varied" binomial distribution looks. There has to be another way.

As our main goal was to decrease the standard deviation of a binomial distribution let us to begin with keep this method and make it available through a function call.

In [162]:
def alberti_distribution(n, sigma):
    
    mu = n/2
    # The new normal distribution
    new_norm = normal_distribution(mu, sigma)
    # The binomial distribution for n
    bin = binomial_distribution(n)
    # The normal distribution with the binomial distribution's sigma 
    bin_sigma = sqrt(sum(bin(i)*(i-mu)**2 for i in range(n+1)))
    bin_norm = normal_distribution(mu, bin_sigma)
    
    def diff_to_integ(val, lower_bound=False):
        def f(a):
            # Set lower bound depending on whether it is specified or not
            lower = val-a if not lower_bound else lower_bound
            # Return value of integral minus the corresponding value of the binomial distribution
            return integrate(norm, lower, val+a) - bin(val)

        return f
        
    ## Calculating the interval boundaries
    
    interval_boundaries = []
   
    # Here we have to behave differently depending on whether n is even or odd
    if int(mu) == mu:
        a = solve(diff_to_integ(ceil(mu)), 0)[0]
    else:
        a = solve(diff_to_integ(ceil(mu), lower_bound=mu), 0)[0]
        interval_boundaries.append(mu)
    interval_boundaries.append(ceil(mu)+a)
    
    # Calculate the rest of the boundaries
    for i in range(int(ceil(mu))+1, n):
        lower_bound = interval_boundaries[-1]
        a = solve(diff_to_integ(i, lower_bound=lower_bound), 0)[0]
        interval_boundaries.append(i+a)
    
    # Mirroring for the other side
    other_side = n - arr(interval_boundaries)[::-1]
    # Another moment where a differentiation between even and odd n is to be made
    if int(mu) == mu:
        interval_boundaries = np.concatenate((other_side, interval_boundaries))
    else:
        interval_boundaries = np.concatenate((other_side, interval_boundaries[1::]))
    # Adding +/- infinity to the list
    interval_boundaries = list(interval_boundaries)
    interval_boundaries.insert(0, -inf)
    interval_boundaries.append(inf)
       
    # Calculating the corresponding integrals
    vals = []
    for i in range(n+1):
        vals.append(integrate(new_norm, interval_boundaries[i], interval_boundaries[i+1])[0])
    vals = arr(vals)

    def f(x):
        return vals[x]
    
    return f

In [163]:
p = fig()
dist1 = alberti_distribution(20, 10)
dist2 = alberti_distribution(20, 5)
dist3 = binomial_distribution(20)
x_vals = arr(range(21))
p.vbar(x_vals, top=dist3(x_vals), width=0.8)
p.line(x_norm, dist1(round(x_norm)), color="red")
show(p)

TypeError: only length-1 arrays can be converted to Python scalars